In [1]:
%matplotlib inline
%matplotlib widget 

import numpy as np
from scipy.spatial.transform import Rotation as R
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import queue
import open3d as o3d
import time
import threading

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
hand_lmks_file = np.load('../hand_landmarks.npz')

In [3]:
hand_lmks = hand_lmks_file["landmarks"]

In [4]:
hand_lmks.shape

(67, 21, 3)

In [5]:
hand_lmks_queue = queue.Queue()

In [6]:
for i in range(hand_lmks.shape[0]):
    lmks = hand_lmks[i, ...]
    wrist, all_fingers = lmks[0, :], lmks[1:, :]
    all_finges = np.reshape(all_fingers, (5, 4, 3))
    hand_lmks_queue.put((wrist, all_fingers))

In [7]:
hand_lmks_queue.qsize()

67

In [8]:
def calculate_angles_between_joints(wrist_XYZ, fingers_XYZ_wrt_wrist, degrees=True):
    def angle_between(a, b, project_to=None):
        """
        project_to = None means project to "xyz"
        """

        if a.ndim == 1:
            a = np.full_like(b, a)

        mask = [1, 1, 1]
        if project_to is not None:
            assert project_to in ["xy", "xz", "yz"]
            if project_to == "xy":
                #mask = [1, 1, 0]
                a = np.delete(a, -1, axis=1)
                b = np.delete(b, -1, axis=1)
            elif project_to == "xz":
                ##mask = [1, 0, 1]
                a = np.delete(a, 1, axis=1)
                b = np.delete(b, 1, axis=1)
            else:
                #mask = [0, 1, 1]
                a = np.delete(a, 0, axis=1)
                b = np.delete(b, 0, axis=1)
        #a = a * mask
        #b = b * mask

        dot_product = np.sum(a * b, axis=1)  # calculate dot product by element-wise style instead of using np.dot
        magnitude_a = np.linalg.norm(a, axis=1)
        magnitude_b = np.linalg.norm(b, axis=1)
        cos_theta = dot_product / (magnitude_a * magnitude_b)
        angle_radians = np.arccos(cos_theta)
        angle_degrees = np.degrees(angle_radians)

        # Get the direction
        M = np.concatenate([a[:, None, :], b[:, None, :]], axis=1)
        dets = np.linalg.det(M)
        directions = np.sign(dets)
        angle_degrees *= directions

        return angle_degrees

    assert np.sum(np.abs(wrist_XYZ)).astype(np.int8) == 0

    angles = np.zeros(shape=fingers_XYZ_wrt_wrist.shape[:-1])
    y_unit = np.array([0, 1, 0])

    """
    For now, we dont calculate the angles of thumb finger
    """

    # Angles of J11 - > J51
    # The order of params a and b is important here, because we will compute determinant to get the direction
    angles[:, 0] = angle_between(vector_y,
                                    fingers_XYZ_wrt_wrist[:, 1, :] - fingers_XYZ_wrt_wrist[:, 0, :],
                                    project_to="yz")

    # Angles of J12 - > J52
    # The order of params a and b is important here, because we will compute determinant to get the direction
    angles[:, 1] = angle_between(fingers_XYZ_wrt_wrist[:, 0, :],
                                    fingers_XYZ_wrt_wrist[:, 1, :] - fingers_XYZ_wrt_wrist[:, 0, :],
                                    project_to="xy")

    # Angles of J13 - > J53
    # The order of params a and b is important here, because we will compute determinant to get the direction
    angles[:, 2] = angle_between(fingers_XYZ_wrt_wrist[:, 1, :] - fingers_XYZ_wrt_wrist[:, 0, :],
                                    fingers_XYZ_wrt_wrist[:, 2, :] - fingers_XYZ_wrt_wrist[:, 1, :],
                                    project_to="xy")

    # Angles of J14 - > J54
    # The order of params a and b is important here, because we will compute determinant to get the direction
    angles[:, 3] = angle_between(fingers_XYZ_wrt_wrist[:, 2, :] - fingers_XYZ_wrt_wrist[:, 1, :],
                                    fingers_XYZ_wrt_wrist[:, 3, :] - fingers_XYZ_wrt_wrist[:, 2, :],
                                    project_to="xy")

    joint_1_weight = np.interp(np.absolute(angles[:, 1]), [0, 90], [1, 0])
    angles[:, 0] *= joint_1_weight

    #angles = bound_angles(angles, degrees=True)

    if not degrees:
        angles = angles * math.pi / 180                                     
    return angles

In [9]:
def show_lmks(hand_lmks_queue):
    x = np.array([[500, 0, 0],
                  [0, 0, 0]])
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(x)

    lines = [[0, 0]]
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(x),
        lines=o3d.utility.Vector2iVector(lines)
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)    
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.add_geometry(line_set)

    for i in range(hand_lmks_queue.qsize()):
        wrist, finger_lmks = hand_lmks_queue.get()
        finger_lmks_flat = np.reshape(finger_lmks, (-1, 3))
        pts = np.vstack((wrist[None, :], finger_lmks_flat))
        pcd.points = o3d.utility.Vector3dVector(pts[:5, :])

        lines = [[0, 1], [1, 2], [2, 3], [3, 4]]
        colors = [[1, 0, 0] for i in range(len(lines))]
        line_set.points = o3d.utility.Vector3dVector(pts)  # Update the points
        line_set.lines = o3d.utility.Vector2iVector(lines)  # Update the lines
        line_set.colors = o3d.utility.Vector3dVector(colors)

        # Update the visualization
        vis.update_geometry(pcd)
        vis.update_geometry(line_set)
        vis.poll_events()
        vis.update_renderer()
        
        # Capture screen image
        vis.capture_screen_image("temp_%04d.jpg" % i)
        
        # Calc. angles
        finger_lmks = np.reshape(finger_lmks, (5, 4, 3))
        angles = calculate_angles_between_joints(wrist, finger_lmks)
        
        print("--- Frame {}: ---".format(i))
        print(angles[0, :])
        
        time.sleep(0.1)

In [10]:
show_thread = threading.Thread(target=show_lmks, args=(hand_lmks_queue,), daemon=True)
show_thread.start()

--- Frame 0: ---
[13.41328684 -3.77346185 -5.61133709  0.61858127]
--- Frame 1: ---
[12.03187402 -9.88365337 -6.95293542 -3.3093726 ]
--- Frame 2: ---
[12.0062507  -8.37070997 -6.54351086 -1.98267007]
--- Frame 3: ---
[11.80069102 -7.60169358 -6.25279916  2.21684885]
--- Frame 4: ---
[11.46645601 -7.10262022 -5.99323215  2.56355627]
--- Frame 5: ---
[11.53587713 -8.14796192 -6.15641606  1.20415029]
--- Frame 6: ---
[13.5866985  -7.01710656 -5.81100607 -7.8983707 ]
--- Frame 7: ---
[25.36256452  3.18104939  0.15396645  2.97940182]
--- Frame 8: ---
[27.77077108  4.18192967  4.74312909  2.44397583]
--- Frame 9: ---
[27.70463314  4.355189    2.77906683  2.63087691]
--- Frame 10: ---
[27.41531457  4.74303118  1.5517541  -1.74983372]
--- Frame 11: ---
[27.90031679  4.24266333 -0.96991582 -4.08254913]
--- Frame 12: ---
[ 28.09365051   5.19435567   0.12239566 -10.48787283]
--- Frame 13: ---
[ 27.80098158   5.66489237  -0.2706732  -17.60053745]
--- Frame 14: ---
[ 28.0574014    5.31774126  -0.6

In [17]:
landmarks = hand_lmks[15, :]

In [10]:
landmarks.shape

(21, 3)

In [37]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(landmarks)

lines = [[0, 1], [1, 2], [2, 3], [3, 4],
                     [0, 5], [5, 6], [6, 7], [7, 8],
                     [0, 9], [9, 10], [10, 11], [11, 12],
                     [0, 13], [13, 14], [14, 15], [15, 16],
                     [0, 17], [17, 18], [18, 19], [19, 20]]    
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(landmarks),
    lines=o3d.utility.Vector2iVector(lines)
)
o3d.visualization.draw_geometries([line_set, pcd])

In [18]:
wrist, fingers = landmarks[0, ...], landmarks[1:, ...]
fingers = np.reshape(fingers, (5, 4, 3))

In [83]:
thumb = fingers[0, ...]

In [84]:
thumb.shape

(4, 3)

In [85]:
x_unit = np.array([1, 0, 0])
y_unit = np.array([0, 1, 0])
z_unit = np.array([0, 0, 1])

In [86]:
dot_j11_and_x = np.dot(x_unit, (thumb[1, ...] - thumb[0, ...]) * [1, 0, 1])
cos_value = dot_j11_and_x / (np.linalg.norm((thumb[1, ...] - thumb[0, ...]) * [1, 0, 1]) * np.linalg.norm(x_unit))
angle_radian = np.arccos(cos_value)
J11_angle = np.degrees(angle_radian)

In [87]:
J11_angle

79.54312753152918

In [88]:
dot_j12_and_z = np.dot(z_unit, (thumb[2, ...] - thumb[1, ...]) * [0, 1, 1])
cos_value = dot_j12_and_z / (np.linalg.norm((thumb[2, ...] - thumb[1, ...]) * [0, 1, 1]) * np.linalg.norm(z_unit))
angle_radian = np.arccos(cos_value)
J12_angle = np.degrees(angle_radian)

In [89]:
J12_angle

68.46304998948226

In [90]:
dot_j13 = np.dot(thumb[1, ...] - thumb[0, ...], thumb[2, ...] - thumb[1, ...])
cos_value = dot_j13 / (np.linalg.norm(thumb[1, ...] - thumb[0, ...]) * np.linalg.norm(thumb[2, ...] - thumb[1, ...]))
angle_rad = np.arccos(cos_value)
J13_angle = np.degrees(angle_rad)

In [91]:
J13_angle

8.54483050428159

In [95]:
np.zeros(4)

array([0., 0., 0., 0.])

In [96]:
thumb

array([[  5.46387672,  28.65561281,  30.16236796],
       [  9.884449  ,  70.96576297,  54.11424478],
       [ 14.62564868, 100.8227585 ,  65.89747618],
       [ 20.3321516 , 109.31971416,  56.97020626]])

In [97]:
a = np.array([x_unit, z_unit, thumb[1, ...] - thumb[0, ...]])

In [98]:
b = np.array([(thumb[1, ...] - thumb[0, ...]) * [1, 0, 1],
              (thumb[2, ...] - thumb[1, ...]) * [0, 1, 1] ,
              thumb[2, ...] - thumb[1, ...]])

In [99]:
x = np.arccos(np.sum(a * b, axis=1) / (np.linalg.norm(a, axis=1) * np.linalg.norm(b, axis=1)))

In [100]:
y = np.degrees(x)

In [101]:
y = np.concatenate([y, y[-1:]])

In [102]:
y

array([79.54312753, 68.46304999,  8.5448305 ,  8.5448305 ])

In [103]:
np.linalg.det(np.array([[1, 2], [3, 4]]))

-2.0000000000000004

In [106]:
np.cross(np.array([[1, 2, 3], [3, 4, 2]]))

TypeError: cross() missing 1 required positional argument: 'b'

In [107]:
np.cross([1, 2, 3], [3, 4, 2])

array([-8,  7, -2])

In [110]:
np.linalg.det([[5, 0], [6, -2]])

-9.999999999999998

In [111]:
np.cross([1, 1, 1], [1, 1, 1])

array([0, 0, 0])

In [113]:
5 * -1100 / np.absolute(1100)

-5.0

In [114]:
a = np.array([[1, 0, 0],
              [1, 0, 0]])
b = np.array([[0, 1, 0],
              [0, -1, 0]])
np.cross(a, b)

array([[ 0,  0,  1],
       [ 0,  0, -1]])

In [120]:
np.round(1 / (0 * 1e-9))

ZeroDivisionError: float division by zero

In [146]:
c_z = 0

In [147]:
c_z = c_z + 1e-9

In [148]:
round(5 * c_z / np.absolute(c_z), 4)

5.0

In [152]:
898 * 1e-9 / 1e-9

898.0

In [162]:
x = np.array([1, 2, 3])
y = np.array([-1])

In [163]:
x[-1] = x[-1] * y[0]

In [164]:
x

array([ 1,  2, -3])